In [1]:
with open('17.txt', 'r') as file:
    data = [int(s) for s in file.read().split(',')]


In [2]:
class Halt(Exception):
    pass


class Yield(Exception):
    pass


class VM:
    def __init__(self, memory, inputs, outputs):
        self.ops = {1: self.add, 
                    2: self.mul, 
                    3: self.input_, 
                    4: self.output, 
                    5: self.jnz,
                    6: self.jz,
                    7: self.lt,
                    8: self.eq,
                    9: self.rbo,
                    99: self.hcf}
        self.memory = memory
        self.pointer = 0
        self.base = 0
        self.inputs = inputs
        self.outputs = outputs

    def run(self):
        while self.step():
            pass
        
    def step(self):
        try:
            while True:
                self.ops[self.read(self.pointer) % 100]()
        except Yield:
            return True
        except Halt:
            return False

    def add(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) + self.read(b))

    def mul(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) * self.read(b))

    def input_(self):
        a, = self.decode(1)
        self.write(a, self.inputs.pop(0))

    def output(self):
        a, = self.decode(1)
        self.outputs.append(self.read(a))
        raise Yield()

    def jnz(self):
        a, b = self.decode(2)
        if self.read(a) != 0:
            self.pointer = self.read(b)

    def jz(self):
        a, b = self.decode(2)
        if self.read(a) == 0:
            self.pointer = self.read(b)

    def lt(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) < self.read(b) else 0)

    def eq(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) == self.read(b) else 0)
        
    def rbo(self):
        a, = self.decode(1)
        self.base += self.read(a)

    def hcf(self):
        raise Halt()

    def decode(self, count):
        mode = self.read(self.pointer) // 100
        self.pointer += 1

        params = []
        for i in range(count):
            value = self.pointer
            if mode % 10 == 0:
                value = self.read(value)
            elif mode % 10 == 2:
                value = self.base + self.read(value)
            params.append(value)
            self.pointer += 1
            mode //= 10
        return params
    
    def read(self, offset):
        self.extend(offset)
        return self.memory[offset]
    
    def write(self, offset, value):
        self.extend(offset)
        self.memory[offset] = value
    
    def extend(self, offset):
        if offset >= len(self.memory):
            self.memory.extend([0] * (offset - len(self.memory) + 1))


In [40]:
ORIENT = {('^', '>'): '>',
          ('v', '<'): '>',
          ('v', '>'): '<',
          ('^', '<'): '<',
          ('<', '>'): '^',
          ('>', '<'): '^',
          ('>', '>'): 'v', 
          ('<', '<'): 'v'}
MOVE: {'^': lambda pos: (pos[0] - 1, pos[1]),
       'v': lambda pos: (pos[0] + 1, pos[1]),
       '<': lambda pos: (pos[0], pos[1] - 1),
       '>': lambda pos: (pos[0], pos[1] + 1)}

scaf = set()
i, j = start
prev = None
align = 0
while pos != prev:
    prev = i, j
    if (i - 1, j) not in scaf:
        while i - 1 >= 0 and map_[i - 1][j] == '#':
            i -= 1
            if (i, j) in scaf: align += i * j
            scaf.add((i, j))
    if (i + 1, j) not in scaf:
        while i + 1 < len(map_) and map_[i + 1][j] == '#':
            i += 1
            if (i, j) in scaf: align += i * j
            scaf.add((i, j))
    if (i, j - 1) not in scaf:
        while j - 1 >= 0 and map_[i][j - 1] == '#':
            j -= 1
            if (i, j) in scaf: align += i * j
            scaf.add((i, j))
    if (i, j + 1) not in scaf:
        while j + 1 < len(map_[i]) and map_[i][j + 1] == '#':
            j += 1
            if (i, j) in scaf: align += i * j
            scaf.add((i, j))

align


8520

In [77]:
ORIENT = {('^', '>'): '>',
          ('v', '<'): '>',
          ('v', '>'): '<',
          ('^', '<'): '<',
          ('<', '>'): '^',
          ('>', '<'): '^',
          ('>', '>'): 'v', 
          ('<', '<'): 'v'}
MOVE = {'^': lambda pos: (pos[0] - 1, pos[1]),
        'v': lambda pos: (pos[0] + 1, pos[1]),
        '<': lambda pos: (pos[0], pos[1] - 1),
        '>': lambda pos: (pos[0], pos[1] + 1)}
TURN = {'<': 'L', '>': 'R'}

def read(pos):
    i, j = pos
    if i >= 0 and i < len(map_):
        if j >= 0 and j < len(map_[i]):
            return map_[i][j]
    return '.'
    
    
scaf = set()
pos = start
orient = read(start)
prev = None
align = 0
program = ''
while pos != prev:
    prev = pos
    for turn in '<>':
        reorient = ORIENT[(orient, turn)]
        move = MOVE[reorient](pos)
        if move not in scaf:
            c = 0
            while read(move) == '#':
                c += 1
                orient = reorient
                pos = move
                if pos in scaf: align += pos[0] * pos[1]
                scaf.add(pos)
                move = MOVE[orient](pos)
            if c > 0:
                program += f'{TURN[turn]}{c}'

print(program)
print(align)


R6L8R8R6L8R8R4R6R6R4R4L8R6L10L10R4R6R6R4R4L8R6L10L10R4R6R6R4R4L8R6L10L10R6L8R8L8R6L10L10
8520


In [76]:
inputs = []
outputs = []
vm = VM(data.copy(), inputs, outputs)
vm.memory[0] = 2

program = """A,A,B,C,B,C,B,C,A,C
R,6,L,8,R,8
R,4,R,6,R,6,R,4,R,4
L,8,R,6,L,10,L,10
n
"""
inputs += [ord(c) for c in program]

vm.run()

outputs[-1]


926819